In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
from langchain.llms import OpenAI
import os
from langchain.chains import SequentialChain


load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [ ]:
MODEL = "gpt-4-1106-preview"

OUTPUT_FOLDER = "guis"
DATASET = "dataset.csv"

df = pd.read_csv(DATASET, sep=";")

In [ ]:
from prompts import SUMMARY2REQUIREMENTS_ZS, REQUIREMENTS2IMPLEMENTATION_ZS, IMPLEMETATION2STRUCTURE_ZS, STRUCTURE2CODE_ZS

def s2r2e2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIREMENTS_ZS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_ZS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_ZS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))


In [ ]:
from prompts import SUMMARY2REQUIRMENTS_FS, REQUIREMENTS2IMPLEMENTATION_FS, IMPLEMETATION2STRUCTURE_FS, STRUCTURE2CODE_FS

def s2r2e2s2c_fs (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIRMENTS_FS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_FS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_FS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [ ]:
from prompts import SUMMARY2STRUCTURE_ZS, STRUCTURE2CODE_ZS

def s2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2STRUCTURE_ZS, output_key="structure",)
    chain_2 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code",)

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [ ]:
from prompts import SUMMARY2STRUCTURE_FS, STRUCTURE2CODE_FS

def s2s2c_fs (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2STRUCTURE_FS, output_key="structure",)
    chain_2 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code",)

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [ ]:
from prompts import SUMMARY2CODE_ZS

def s2c(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [ ]:
from prompts import CoT_ZS

def s2c_xot_zs(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=CoT_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [ ]:
from prompts import CoT_FS

def s2c_xot_fs(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=CoT_FS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [ ]:
import re

def extract_html(code, file, folder):
    html_content = re.search(r'<!DOCTYPE html>.*</html>', code, re.DOTALL)
    extracted_text = html_content.group()
    try:
        with open(f"{folder}/{file}.html", 'w') as f:
            f.write(extracted_text)
    except Exception as e:
        print(f"{file} failed")
        print(e)

In [ ]:
def prototyping(summary,folder):
   
    code = s2c(summary)
    extract_html(code,"instruction",folder)    
    
    code = s2c_xot_zs(summary)
    extract_html(code,"CoT_zs",folder)    
    
    code = s2c_xot_fs(summary)
    extract_html(code,"CoT_fs",folder)    

    code = s2s2c(summary)
    extract_html(code,"simple_pd_zs",folder)    

    code = s2s2c_fs(summary)
    extract_html(code,"simple_pd_fs",folder)    

    code = s2r2e2s2c(summary)
    extract_html(code,"complex_pd_zs",folder)    

    code = s2r2e2s2c_fs(summary)
    extract_html(code,"complex_pd_fs",folder)    
   


In [ ]:
def pipeline():
    for index, row in df.iterrows():
        os.makedirs(f"{OUTPUT_FOLDER}/{row['UI Number']}")
        prototyping(row['Summary'],f"{OUTPUT_FOLDER}/{row['UI Number']}")


In [ ]:
pipeline()